In [1]:
# alpha: initial import statements
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split as tts
from sklearn.preprocessing import minmax_scale as mms
# !pip install category_encoders
# import category_encoders as ce
# from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
# from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
from sklearn.svm.libsvm import predict
import random

In [2]:
# bravo: give background, then load and inspect data
# here's a URL for background: https://www.kaggle.com/t/32b89e93d8a44743983a0ab1c19c85f3
# Q: "Can you predict which [Tanzanian] water pumps are faulty?"
# H_null: something like "all features are equally likely to make H2O unclean and/or non-potable"
# H_one: akin to "one or a group of features enables high-accuracy prediction of faulty pumps"

kaggle_path = 'C:\\Users\\jhump\\Desktop\\Desktop_professional\\LSDS\\Full_Course\\train_features.csv'

df = pd.read_csv(kaggle_path)
print(df.shape)
df.head()

(59400, 40)


,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,...,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,...,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,...,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,...,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59400 entries, 0 to 59399
Data columns (total 40 columns):
id                       59400 non-null int64
amount_tsh               59400 non-null float64
date_recorded            59400 non-null object
funder                   55765 non-null object
gps_height               59400 non-null int64
installer                55745 non-null object
longitude                59400 non-null float64
latitude                 59400 non-null float64
wpt_name                 59400 non-null object
num_private              59400 non-null int64
basin                    59400 non-null object
subvillage               59029 non-null object
region                   59400 non-null object
region_code              59400 non-null int64
district_code            59400 non-null int64
lga                      59400 non-null object
ward                     59400 non-null object
population               59400 non-null int64
public_meeting           56066 non-null object
r

In [4]:
# charlie: check data for nan's
df.isna().sum()

id                           0
amount_tsh                   0
date_recorded                0
funder                    3635
gps_height                   0
installer                 3655
longitude                    0
latitude                     0
wpt_name                     0
num_private                  0
basin                        0
subvillage                 371
region                       0
region_code                  0
district_code                0
lga                          0
ward                         0
population                   0
public_meeting            3334
recorded_by                  0
scheme_management         3877
scheme_name              28166
permit                    3056
construction_year            0
extraction_type              0
extraction_type_group        0
extraction_type_class        0
management                   0
management_group             0
payment                      0
payment_type                 0
water_quality                0
quality_

In [5]:
df1 = df.copy()

# source for following: https://stackoverflow.com/questions/22470690/get-list-of-pandas-dataframe-columns-based-on-data-type
df1_dtype_groups = df1.columns.to_series().groupby(df1.dtypes).groups
df1_dtype_groups

{dtype('int64'): Index(['id', 'gps_height', 'num_private', 'region_code', 'district_code',
        'population', 'construction_year'],
       dtype='object'),
 dtype('float64'): Index(['amount_tsh', 'longitude', 'latitude'], dtype='object'),
 dtype('O'): Index(['date_recorded', 'funder', 'installer', 'wpt_name', 'basin',
        'subvillage', 'region', 'lga', 'ward', 'public_meeting', 'recorded_by',
        'scheme_management', 'scheme_name', 'permit', 'extraction_type',
        'extraction_type_group', 'extraction_type_class', 'management',
        'management_group', 'payment', 'payment_type', 'water_quality',
        'quality_group', 'quantity', 'quantity_group', 'source', 'source_type',
        'source_class', 'waterpoint_type', 'waterpoint_type_group'],
       dtype='object')}

In [6]:
# what features will show whether or not H2O is clean and potable?
# let's investigate 'water_quality'
df1.water_quality.unique()  # questions: merge both 'abandoned' strings into one? what to do with 'unknown'?

array(['soft', 'salty', 'milky', 'unknown', 'fluoride', 'coloured',
       'salty abandoned', 'fluoride abandoned'], dtype=object)

In [7]:
# delta: preprocess data
# CREDIT for this function, to Ryan Herr/LSDS

def train_validation_test_split(
    X, y, train_size=0.8, val_size=0.1, test_size=0.1, 
    random_state=None, shuffle=True):
        
    assert train_size + val_size + test_size == 1
    
    X_train_val, X_test, y_train_val, y_test = tts(X, y, test_size=test_size,
                                                   random_state=random_state, shuffle=shuffle)
    
    X_train, X_val, y_train, y_val = tts(X_train_val, y_train_val,
                                         test_size=val_size/(train_size+val_size), 
                                         random_state=random_state, shuffle=shuffle)
    print('X_train is:', X_train, '\n')
    return X_train, X_val, X_test, y_train, y_val, y_test

In [22]:
# echo: fit, validate, and present model

# 2019-02-04 comment: I believe that some sort of advanced regression will be best here, but TBD
# for baseline model, X might include 'id', 'region_code', 'population', 'construction_year'
# IMPORTANT NOTE: 2019-02-06 0830hrs PST: REFACTOR y

train_labels = pd.read_csv('C:\\Users\\jhump\\Desktop\\Desktop_professional\\LSDS\\train_labels.csv')
df2 = train_labels.copy()

# X = df1[['id', 'region_code', 'population', 'construction_year']]
# got much, much lower accuracy score, after refactoring, so will start to 'twiddle' the feature knobs
# first, take out 'id' -> accuracy score: slightly increased, at 0.5579545454545455S
# X = df1[['region_code', 'population', 'construction_year']]

# second, add in 'gps_height' -> accuracy score: not significantly changed
# X = df1[['gps_height', 'region_code', 'population', 'construction_year']]

# third, take 'gps_height' back out, and take out 'region_code' -> accuracy score: 0.5430134680134681
# X = df1[['population', 'construction_year']]
# KEY: thus, need to add in categorical features--onehotencoding as needed

In [23]:
# continue fitting, validating LogReg model
# now, after those 3 trials above, will add 'water_quality' to X -> accuracy score:
X = df1[['population', 'construction_year', 'water_quality']]

# call 'mms' on X after creating X namespace
mms(X)
y = df2.status_group

X_train, X_test, y_train, y_test = tts(X, y, train_size=0.6, random_state=42)

log_reg = LogisticRegression().fit(X, y)
log_reg.fit(X_train, y_train)
y_pred = log_reg.predict(X_test)
accuracy_score(y_test, y_pred)
# pd.get_dummies(df)

# attempt this after 60% accuracy achieved
# train_validation_test_split(X, y)
# pipeline = make_pipeline(
#     ce.OneHotEncoder(use_cat_names=True), 
#     StandardScaler(), 
#     LogisticRegression(solver='lbfgs')
# )

# pipeline.fit(X_train, y_train)
# y_pred = pipeline.predict(X_val)
# accuracy_score(y_val, y_pred)

ValueError: could not convert string to float: 'soft'

In [9]:
# examine log_reg and its methods

In [10]:
# the pseudo-random baseline

print('id', ',', 'status_group')
pumps = X['id']
for pump in pumps[:10]:
    if random.choice(['not_functional', 'functional']) == 'functional':
        print(pump, ',', 'functional')

id , status_group
34310 , functional
19728 , functional
9944 , functional
19816 , functional
54551 , functional


In [11]:
# the simple LogReg baseline for Kaggle csv
# predicted_non_abandoned_pump_count = 0
max_rows = 0

print('id', ',', 'status_group')
pumps = X_test['id']
while max_rows < 14359:
    for pump, pred in zip(pumps, y_pred):
        if pred != 'unknown' and 'abandoned' not in pred and pred != 'salty':
            print(pump, ',', 'functional')
            # predicted_non_abandoned_pump_count += 1
            max_rows += 1

# print('predicted number of unabandoned pumps is:', predicted_non_abandoned_pump_count)
print('number of rows generated is:', max_rows)


id , status_group
37098 , functional
14530 , functional
62607 , functional
46053 , functional
47083 , functional
12465 , functional
12921 , functional
14606 , functional
9417 , functional
71095 , functional
38698 , functional
71514 , functional
73962 , functional
42283 , functional
42806 , functional
645 , functional
45781 , functional
38258 , functional
61113 , functional
72970 , functional
6961 , functional
65808 , functional
58051 , functional
63313 , functional
3654 , functional
47965 , functional
21788 , functional
31842 , functional
2315 , functional
35144 , functional
74213 , functional
62492 , functional
24352 , functional
49025 , functional
45655 , functional
55584 , functional
29761 , functional
64337 , functional
12297 , functional
16755 , functional
7607 , functional
45077 , functional
49878 , functional
32410 , functional
40717 , functional
64889 , functional
16558 , functional
68557 , functional
39601 , functional
35727 , functional
56365 , functional
57163 , functional
1

43808 , functional
26726 , functional
53012 , functional
27921 , functional
53705 , functional
7775 , functional
8799 , functional
4277 , functional
58469 , functional
67994 , functional
2221 , functional
68462 , functional
46972 , functional
44674 , functional
64012 , functional
51110 , functional
43004 , functional
43847 , functional
34684 , functional
6858 , functional
66196 , functional
48456 , functional
59643 , functional
12414 , functional
24092 , functional
38775 , functional
62613 , functional
66168 , functional
50779 , functional
17835 , functional
29114 , functional
33485 , functional
62254 , functional
1502 , functional
42223 , functional
25282 , functional
12466 , functional
61439 , functional
4155 , functional
64157 , functional
30661 , functional
16648 , functional
29111 , functional
10213 , functional
20036 , functional
72609 , functional
30368 , functional
17305 , functional
59461 , functional
12915 , functional
20706 , functional
53212 , functional
74235 , functional


37727 , functional
34250 , functional
14245 , functional
17439 , functional
65883 , functional
73312 , functional
28725 , functional
47698 , functional
55765 , functional
51969 , functional
9759 , functional
72856 , functional
3825 , functional
393 , functional
28875 , functional
43843 , functional
13152 , functional
64404 , functional
62727 , functional
1886 , functional
58038 , functional
49507 , functional
19046 , functional
1888 , functional
69624 , functional
11510 , functional
21560 , functional
74205 , functional
32432 , functional
12410 , functional
53962 , functional
16867 , functional
66090 , functional
44955 , functional
6097 , functional
3500 , functional
54057 , functional
61473 , functional
21117 , functional
61730 , functional
58317 , functional
70521 , functional
17359 , functional
34758 , functional
12804 , functional
41468 , functional
25689 , functional
70811 , functional
59182 , functional
60030 , functional
50544 , functional
50843 , functional
31214 , functional
1

58416 , functional
66424 , functional
29411 , functional
9601 , functional
17348 , functional
69060 , functional
59200 , functional
60560 , functional
18961 , functional
66989 , functional
136 , functional
25178 , functional
11639 , functional
61412 , functional
23385 , functional
1559 , functional
44255 , functional
11441 , functional
49483 , functional
50348 , functional
58360 , functional
62256 , functional
52285 , functional
43184 , functional
61070 , functional
52087 , functional
63316 , functional
19987 , functional
17864 , functional
60226 , functional
67542 , functional
19810 , functional
16815 , functional
38075 , functional
32480 , functional
5781 , functional
9926 , functional
59376 , functional
64820 , functional
65666 , functional
3504 , functional
66988 , functional
62034 , functional
14964 , functional
11697 , functional
9290 , functional
48556 , functional
51899 , functional
50610 , functional
7225 , functional
44923 , functional
21496 , functional
41138 , functional
36

59416 , functional
57291 , functional
72317 , functional
34533 , functional
32782 , functional
34835 , functional
56075 , functional
44262 , functional
23533 , functional
17536 , functional
56450 , functional
42409 , functional
46625 , functional
66373 , functional
22053 , functional
24831 , functional
60552 , functional
60412 , functional
51723 , functional
12251 , functional
73804 , functional
32900 , functional
30052 , functional
26525 , functional
20035 , functional
9347 , functional
57317 , functional
15590 , functional
72534 , functional
10904 , functional
20008 , functional
28293 , functional
17186 , functional
19926 , functional
13662 , functional
35775 , functional
64994 , functional
69765 , functional
18068 , functional
15193 , functional
37360 , functional
41973 , functional
18084 , functional
38531 , functional
398 , functional
62999 , functional
23832 , functional
2029 , functional
45769 , functional
54035 , functional
1325 , functional
29097 , functional
42017 , functiona

...

72610 , functional
28390 , functional
59076 , functional
21534 , functional
71676 , functional
70550 , functional
64549 , functional
56615 , functional
64072 , functional
36995 , functional
34922 , functional
56629 , functional
60977 , functional
19093 , functional
59677 , functional
33421 , functional
66712 , functional
9300 , functional
72475 , functional
7704 , functional
9825 , functional
21989 , functional
71110 , functional
31543 , functional
66390 , functional
48952 , functional
60367 , functional
6671 , functional
20595 , functional
62115 , functional
66325 , functional
40615 , functional
26163 , functional
24412 , functional
41724 , functional
11139 , functional
7159 , functional
38857 , functional
8684 , functional
9291 , functional
1959 , functional
16446 , functional
50905 , functional
28595 , functional
7144 , functional
4906 , functional
28612 , functional
65128 , functional
49938 , functional
2346 , functional
43216 , functional
7450 , functional
43218 , functional
34515

40226 , functional
17424 , functional
54043 , functional
18723 , functional
33683 , functional
12262 , functional
58850 , functional
1497 , functional
56489 , functional
73031 , functional
47407 , functional
25846 , functional
44306 , functional
58314 , functional
1905 , functional
58395 , functional
13708 , functional
1417 , functional
46351 , functional
5605 , functional
7347 , functional
45267 , functional
22319 , functional
8758 , functional
49205 , functional
9363 , functional
35277 , functional
68651 , functional
46103 , functional
32259 , functional
721 , functional
27304 , functional
36548 , functional
42639 , functional
28597 , functional
51983 , functional
55188 , functional
48865 , functional
55360 , functional
55828 , functional
66713 , functional
36845 , functional
28220 , functional
3921 , functional
48410 , functional
39751 , functional
63845 , functional
40546 , functional
64650 , functional
52537 , functional
15537 , functional
13724 , functional
23118 , functional
662

65838 , functional
66394 , functional
13855 , functional
46008 , functional
9289 , functional
11287 , functional
37791 , functional
18325 , functional
2323 , functional
13747 , functional
22107 , functional
41474 , functional
7463 , functional
70006 , functional
31273 , functional
22944 , functional
30012 , functional
41564 , functional
2992 , functional
9630 , functional
28648 , functional
52901 , functional
21156 , functional
56970 , functional
17937 , functional
51485 , functional
59919 , functional
58246 , functional
11209 , functional
67890 , functional
12375 , functional
45599 , functional
68964 , functional
40674 , functional
15697 , functional
52120 , functional
54464 , functional
66247 , functional
62051 , functional
72548 , functional
50636 , functional
7505 , functional
21378 , functional
69432 , functional
24606 , functional
5649 , functional
7675 , functional
19666 , functional
40940 , functional
35869 , functional
27689 , functional
34187 , functional
59055 , functional
7

5518 , functional
64237 , functional
34931 , functional
55055 , functional
7198 , functional
69123 , functional
67922 , functional
73180 , functional
54797 , functional
64831 , functional
48974 , functional
28788 , functional
69380 , functional
14229 , functional
36629 , functional
47027 , functional
47856 , functional
22110 , functional
21460 , functional
73576 , functional
42261 , functional
27142 , functional
39531 , functional
63577 , functional
46567 , functional
40926 , functional
53306 , functional
50919 , functional
20402 , functional
38 , functional
25665 , functional
30109 , functional
26486 , functional
47494 , functional
23346 , functional
21196 , functional
6233 , functional
38850 , functional
72267 , functional
19978 , functional
1865 , functional
61406 , functional
44025 , functional
38987 , functional
72651 , functional
29403 , functional
36546 , functional
32605 , functional
52147 , functional
32828 , functional
51699 , functional
47649 , functional
28672 , functional


21343 , functional
28085 , functional
71576 , functional
71584 , functional
23905 , functional
28560 , functional
65853 , functional
39918 , functional
27164 , functional
60056 , functional
3734 , functional
50683 , functional
63707 , functional
22475 , functional
49324 , functional
6438 , functional
68211 , functional
58890 , functional
7348 , functional
55857 , functional
17231 , functional
24598 , functional
45601 , functional
48927 , functional
68220 , functional
3495 , functional
37912 , functional
65416 , functional
14366 , functional
12648 , functional
4799 , functional
9527 , functional
15919 , functional
5079 , functional
28160 , functional
71021 , functional
32602 , functional
24755 , functional
16476 , functional
73592 , functional
39513 , functional
48481 , functional
11133 , functional
26691 , functional
21866 , functional
70250 , functional
21792 , functional
49873 , functional
27468 , functional
60616 , functional
23827 , functional
65371 , functional
21077 , functional


In [12]:
# for further modeling

# faulty_pumps = predict(df1.id, kernel='sigmoid', degree=1, coef0=log_reg.coef_)
# faulty_pumps

In [13]:
# write LogReg baseline data to csv on local machine, in preparation to push to GitHub then Kaggle
prediction_csv_path = 'C:\\Users\\jhump\\Desktop\\Desktop_professional\\LSDS\\Full_Course\\kaggle-prediction.csv'

with open(prediction_csv_path, 'w') as infile:
    max_rows = 0
    infile.write('id' + ', ' + 'status_group' + '\n')
    pumps = X_test['id']
    while max_rows < 14359:
        for pump, pred in zip(pumps, y_pred):
            if 'abandoned' not in pred and 'unknown' not in pred:
                infile.write(str(pump) + ', ' + 'functional' + '\n')
                max_rows += 1
    infile.close()


In [14]:
# attempt to understand tanzania-data-kaggle.csv vis a vis Kaggle submission req's
# command line syntax CREDIT to Ryan Herr/LSDS
tanzania_data_kaggle = 'C:\\Users\\jhump\\Desktop\\Desktop_professional\\LSDS\\Full_Course\\tanzania-data-kaggle.csv'


In [15]:
# 2019-02-06 Based on feedback from Ryan Herr/LSDS, will refactor y above

In [16]:
# foxtrot: analyze and interpret model predictions